In [139]:
import numpy as np
import matplotlib.pyplot as plt

import math

import sympy as sp

## Things which are given / can be calculated 

In [140]:
P_d_sp = sp.symbols('P_d')
D_p_sp, D_g_sp = sp.symbols('D_p D_g')
N_p_sp, N_g_sp, N_g_guess_sp = sp.symbols('N_p N_g, N_g_guess') 
C_sp, F_sp = sp.symbols('C F')
n_p_sp, n_g_sp, n_g_guess_sp = sp.symbols('n_p n_g n_g_guess')
VR_sp, VR_guess_sp, v_t_sp = sp.symbols('VR VR_guess v_t')
power_sp, lifetime_sp = sp.symbols('power lifetime')
W_t_sp = sp.symbols('W_t')


#enter given values

uk = sp.nan
all_values = {
    P_d_sp: 12,
    D_p_sp: sp.nan,
    D_g_sp: uk,
    C_sp: sp.nan,
    N_p_sp: 19,
    N_g_sp: uk,
    N_g_guess_sp: uk,
    n_p_sp: 1200,
    n_g_sp: uk,
    n_g_guess_sp: 387,
    VR_sp: sp.nan,
    VR_guess_sp: sp.nan,
    v_t_sp: sp.nan,
    F_sp: sp.nan,
    power_sp: 5,
    lifetime_sp: 15000,
    W_t_sp: sp.nan,
}

# To have in case
# all_values = {
#     P_d: 10,
#     D_p: sp.nan,
#     D_g: sp.nan,
#     C: sp.nan,
#     N_p: 17,
#     N_g: 54,
#     n_p: 1200,
#     n_g: sp.nan,
#     n_g_guess: 387,
#     VR: sp.nan,
#     VR_guess: sp.nan,
#     v_t: sp.nan,
#     F: sp.nan,
#     power: sp.nan,
#     lifetime: sp.nan
# }

all_values, len(all_values)


({P_d: 12,
  D_p: nan,
  D_g: nan,
  C: nan,
  N_p: 19,
  N_g: nan,
  N_g_guess: nan,
  n_p: 1200,
  n_g: nan,
  n_g_guess: 387,
  VR: nan,
  VR_guess: nan,
  v_t: nan,
  F: nan,
  power: 5,
  lifetime: 15000,
  W_t: nan},
 17)

## Things to find from a table

some, like Y,Z can only be found later, tbd how to handle this

In [141]:
J_p = 0.34
J_G = 0.41

I = 0.1

C_p = 2300 #for steel

k_o = 1.5
k_s = 1.0

k_B = 1.0
k_v = 1.25
k_R = 1.0


#Need cycles to find, but these are not calculated until later
#im not sure the best way to handle this
Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922


#for C_ma
cond = "commercial Enclosed".upper()
# cond = "precision".upper()
# cond = "extra".upper()

safety_factor = 1


## Create the equations

In [142]:

eq1 = sp.Eq(VR_guess_sp, n_p_sp / n_g_guess_sp)
eq2 = sp.Eq(N_g_guess_sp, N_p_sp * VR_guess_sp)

eq3 = sp.Eq(VR_sp, N_g_sp / N_p_sp)


eq4 = sp.Eq(n_g_sp, n_p_sp / VR_sp)
eq5 = sp.Eq(D_p_sp, N_p_sp / P_d_sp)
eq6 = sp.Eq(D_g_sp, N_g_sp / P_d_sp)
eq7 = sp.Eq(C_sp, (N_p_sp + N_g_sp) / (2 * P_d_sp))
eq8 = sp.Eq(v_t_sp, np.pi * D_p_sp * n_p_sp / 12)
eq9 = sp.Eq(W_t_sp, 33000 * power_sp / v_t_sp)
eq10 = sp.Eq(F_sp, 12 / P_d_sp)
eq11 = sp.Eq(VR_sp, n_p_sp / n_g_sp)




## Solve the equations 

In [143]:
#creates a thing of all the non-nan values
known_values = {}

for var, value in all_values.items():
    if value is not sp.nan:
        known_values[var] = value


Solve the equations for VR_guess, n_g_guess, N_p, n_g

In [144]:
# Solve equations eq1 and eq2
solutions_eq1_eq2 = sp.solve([eq1, eq2], dict=True)

# Display the solutions
# for solution in solutions_eq1_eq2:
#     for var, expr in solution.items():
#         print(f"{var} = {expr}")


for solution in solutions_eq1_eq2:
        for var, expr in solution.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)





known_values




{P_d: 12,
 N_p: 19,
 n_p: 1200,
 n_g_guess: 387,
 power: 5,
 lifetime: 15000,
 N_g_guess: 7600/129,
 VR_guess: 400/129}

In [145]:

if known_values[N_g_guess_sp] == sp.nan:  
    a = sp.nan
else:
    known_values[N_g_sp] = round(known_values[N_g_guess_sp], 0)
# known_values[N_g_sp]

# Solve equation eq3 with the updated known values

if all_values[n_g_guess_sp] == sp.nan:
    pass

else:

    solution_eq3 = sp.solve(eq3.subs(known_values), dict=True)

    # Update known values with the solution of eq3
    for sol in solution_eq3:
        for var, expr in sol.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values




{P_d: 12,
 N_p: 19,
 n_p: 1200,
 n_g_guess: 387,
 power: 5,
 lifetime: 15000,
 N_g_guess: 7600/129,
 VR_guess: 400/129,
 N_g: 59.0000000000000,
 VR: 3.10526315789474}

#### Solve the equations multiple times bc some of them depend on others

In [147]:
# Iteratively solve equations until all variables are found

run_count = 0
while True:

    previous_known_values = known_values.copy()
    
    # Solve equations with current known values

    #if all_values[n_g_guess_sp] == sp.nan:
    solutions = sp.solve([eq4.subs(known_values), eq5.subs(known_values), eq6.subs(known_values), eq7.subs(known_values), eq8.subs(known_values), eq9.subs(known_values), eq10.subs(known_values)], dict=True)
    
    # else:
    #     solutions = sp.solve([eq4.subs(known_values), eq5.subs(known_values), eq6.subs(known_values), eq7.subs(known_values), eq8.subs(known_values), eq9.subs(known_values), eq10.subs(known_values)], dict=True)

    # Update known values with solutions
    print(len(solutions))
    for solution in solutions:

        for var, expr in solution.items():

            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)    

    # Break if no new values are found
    if known_values == previous_known_values:
        break
    else:
        run_count += 1
        print(f"Run interation: {run_count}")

    if run_count > 5:
        break

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")


1
Run interation: 1
0
P_d = 12
N_p = 19
n_p = 1200
n_g_guess = 387
power = 5
lifetime = 15000
N_g_guess = 7600/129
VR_guess = 400/129
N_g = 59.0000000000000
VR = 3.10526315789474
C = 3.25000000000000
D_g = 4.91666666666667
D_p = 1.58333333333333
F = 1.00000000000000
W_t = 331.712407707319
n_g = 386.440677966102
v_t = 497.418836818383


### Lists each variable and its equation 
not the greatest format 

#### Assign each variable a value, eventually these should all be numbers

In [171]:
# assert(len(known_values) == len(all_values))
len(known_values), len(all_values)
# known_values

for var, value in known_values.items():
    all_values[var] = value


P_d = all_values[P_d_sp]
D_p = all_values[D_p_sp]
D_g = all_values[D_g_sp]
C = all_values[C_sp]
N_p = all_values[N_p_sp]
N_g = all_values[N_g_sp]
n_p = all_values[n_p_sp]
n_g = all_values[n_g_sp]
n_g_guess = all_values[n_g_guess_sp]
VR = all_values[VR_sp]
VR_guess = all_values[VR_guess_sp]
v_t = all_values[v_t_sp]
F = all_values[F_sp]
power = all_values[power_sp]
lifetime = int(all_values[lifetime_sp])
W_t = all_values[W_t_sp]

all_values

{P_d: 12,
 D_p: 1.58333333333333,
 D_g: 4.91666666666667,
 C: 3.25000000000000,
 N_p: 19,
 N_g: 59.0000000000000,
 N_g_guess: 7600/129,
 n_p: 1200,
 n_g: 386.440677966102,
 n_g_guess: 387,
 VR: 3.10526315789474,
 VR_guess: 400/129,
 v_t: 497.418836818383,
 F: 1.00000000000000,
 power: 5,
 lifetime: 15000,
 W_t: 331.712407707319}

### Final assorted Calculations

In [150]:
if F is sp.nan:
    # print(F_sp)
    F = 12 / P_d

assert(F < 2 * D_p)
assert(F > 0.5 * D_p)
F

1.00000000000000

## Calulating correction Factors

### Calculate $C_{pf}$

In [151]:
#C_pf
C_pf = None

if F / D_p < 0.5:
    assert(False)
    #use graph
elif F <= 1.0:
    C_pf = F / (10 * D_p) - 0.025
elif F <= 15:
    C_pf = F / (10 * D_p) - 0.0375 + 0.0125 * F

assert(C_pf is not None)

C_pf

0.0381578947368421

### Calculate $C_{ma}$

In [152]:
#C_ma

assert(cond is not None)

C_ma = None

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F - 1.093 * 10**-4 * F**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F - 0.926 * 10**-4 * F**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F - 0.822 * 10**-4 * F**2

assert(C_ma is not None)
C_ma

0.142690700000000

In [153]:
k_m = 1 + C_ma + C_pf

## Loading Cycles $N_{cp}, N_{cg}$

In [169]:
# Number of cycles

N_cp = 60 * lifetime * n_p
N_cg = 60 * lifetime * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")

N_cp: 1.08e+09, N_cg: 3.48e+8


### *** Implement $Y_{xx}$  and $Z_{xx}$

Put all the graphs here

## Stresses and allowable Stresses

### $S_{tp}$, $S_{tG}$

In [156]:
s_tp = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_B * k_v
s_tp

25921.4645182475

In [157]:
s_tG = s_tp * J_p / J_G
s_tG

21495.8486248882

### $S_{atp}$, $S_{atG}$

In [158]:
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
s_atP_Greater_than

27285.7521244710

In [159]:
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng
s_atG_Greater_than

21495.8486248882

### Calc $S_c$

In [160]:

aa = (W_t * k_o * k_s * k_m  * k_v)

bb = (F * D_p * I)

S_c = C_p * np.sqrt( float( aa / bb)  ) #
S_c

156646.34335194936

### FInd adjusted values of $S_c$

In [161]:
S_acP_Greater_than = S_c * (k_R * safety_factor) / Z_np
S_acP_Greater_than

174439.13513580107

In [162]:
S_acg_Greater_than = S_c * (k_R * safety_factor) / Z_ng
S_acg_Greater_than

169898.42012142012

## Brinell Hardness Calculations

In [163]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB_p = 0
Bending_HB_p = 0

if int(HB_grade) == 1:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 12.8 ) / 0.0773

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 12.8 ) / 0.0773

elif int(HB_grade) == 2:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 16.4 ) / 0.102

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 16.4 ) / 0.102

else:
    assert(False)


print(f"Contact_HB_p: {Contact_HB_p:.2f} psi \nBending_HB_p: {Bending_HB_p:.2f} psi\n")

print(f"Contact_HB_g: {Contact_HB_g:.2f} psi \nBending_HB_g: {Bending_HB_g:.2f} psi")

Contact_HB_p: 451.36 psi 
Bending_HB_p: 187.40 psi

Contact_HB_g: 437.26 psi 
Bending_HB_g: 112.49 psi


### use tables to find the appropriate Material, and enter its brinell hardness

In [168]:
HB = None

### Calculate actaul allowed bending and contact stresses

In [165]:
#too lazy to implement rn

S_acp = None
S_atp = None

S_acg = None
S_atg = None


if int(HB_grade) == 1:
    S_acp = (HB * 0.322 + 29.1 ) * 1000 
    S_atp = (HB * 0.0773 + 12.8 ) * 1000

    S_acg = (HB * 0.322 + 29.1 ) * 1000
    S_atg = (HB * 0.0773 + 12.8 ) * 1000

elif int(HB_grade) == 2:

    S_acp = (HB * 0.349 + 34.3 ) * 1000
    S_atp = (HB * 0.102 + 16.4 ) * 1000

    S_acg = (HB * 0.349 + 34.3 ) * 1000
    S_atg = (HB * 0.102 + 16.4 ) * 1000
    
else:
    assert(False)


### Calculate Safety factor

In [166]:
contact_SF_p = S_acp * Y_np / (S_c * k_R)
bending_SF_p = S_atp * Y_np / (s_tp * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")

contact_SF_g = S_acg * Y_np / (S_c * k_R)
bending_SF_g = S_atg * Y_np / (s_tG * k_R)

print(f"Contact Safety Factor (Gear): {contact_SF_g:.2f} \nBending Safety Factor (Gear): {bending_SF_g:.2f}")



Contact Safety Factor (Pinion): 1.076724 
Bending Safety Factor (Pinion): 1.77511324514896

Contact Safety Factor (Gear): 1.08 
Bending Safety Factor (Gear): 2.14


## Power transmitting Capacity

In [167]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acP_Greater_than * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")

P_cap = 5.0020 HP
